In [1]:
from keras.models import load_model

model = load_model('face_recog_model.h5')

In [2]:
import cv2
import numpy as np
from mtcnn import MTCNN

detector = MTCNN()
cap = cv2.VideoCapture(0)
classes = ['Pond', 'Blue', 'Tar', 'New']
def extract_face(image):
    faces = detector.detect_faces(image)
    if len(faces) == 0:
        return None, None
    x1, y1, width, height = faces[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    return faces[0]['box'],image[y1:y2, x1:x2]

def preprocess(image):
    box,image = extract_face(image)
    if image is None:
        return None, None
    image = image.astype('float32')/255.0
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
    return box,image

while True:
    ret, frame = cap.read()
    if ret == False:
        break
    box,image = preprocess(frame)
    if image is None or box is None:
        continue
    
    preds = model.predict(image.reshape(1, 224, 224, 3))
    x = np.argmax(preds, axis=1)[0]
    # draw box and write class name
    cv2.rectangle(frame, (box[0], box[1]), (box[0]+box[2], box[1]+box[3]), (0, 255, 0), 2)
    # put pred proba
    txt = ''
    for i in range(4):
        txt += classes[i] + ': ' + str(round(preds[0][i], 2)) + ' '
    cv2.putText(frame, txt, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, classes[x], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 40ms/step
